In [1]:
import os 
import sys 
import time 
import numpy as np
import json

from pyspark.sql import SparkSession 
from pyspark.conf import SparkConf

from pyspark.context import SparkContext
from pyspark.sql.types import (
    DoubleType, LongType, StringType, StructField, StructType)

import platform,socket,re,uuid,json,psutil,logging


# Schemas for all table types here. These should be in separate scripts when
# refactoring code.
CUSTOMER_SCHEMA = StructType([
    StructField("c_custkey", LongType()),
    StructField("c_name", StringType()),
    StructField("c_address", StringType()),
    StructField("c_nationkey", LongType()),
    StructField("c_phone", StringType()),
    StructField("c_acctbal", DoubleType()),
    StructField("c_mktsegment", StringType()),
    StructField("c_comment", StringType()),
])

LINEITEM_SCHEMA = StructType([
    StructField("l_orderkey", LongType()),  
    StructField("l_partkey", LongType()),
    StructField("l_suppkey", LongType()),
    StructField("l_linenumber", LongType()),
    StructField("l_quantity", DoubleType()),
    StructField("l_extendedprice", DoubleType()),
    StructField("l_discount", DoubleType()),
    StructField("l_tax", DoubleType()),
    StructField("l_returnflag", StringType()),
    StructField("l_linestatus", StringType()),
    StructField("l_shipdate", StringType()),
    StructField("l_commitdate", StringType()),
    StructField("l_receiptdate", StringType()),
    StructField("l_shipinstruct", StringType()),
    StructField("l_shipmode", StringType()),
    StructField("l_comment", StringType())
])

NATION_SCHEMA = StructType([
    StructField("n_nationkey", LongType()), 
    StructField("n_name", StringType()),
    StructField("n_regionkey", LongType()),
    StructField("n_comment", StringType()),
])

ORDER_SCHEMA = StructType([
    StructField("o_orderkey", LongType()),
    StructField("o_custkey", LongType()),
    StructField("o_orderstatus", StringType()),
    StructField("o_totalprice", DoubleType()),
    StructField("o_orderdate", StringType()),
    StructField("o_orderpriority", StringType()),
    StructField("o_clerk", StringType()),
    StructField("o_shippriority", LongType()),
    StructField("o_comment", StringType())
])

PART_SCHEMA = StructType([
    StructField("p_partkey", LongType()),    
    StructField("p_name", StringType()),
    StructField("p_mfgr", StringType()),
    StructField("p_brand", StringType()),
    StructField("p_type", StringType()),
    StructField("p_size", LongType()),
    StructField("p_container", StringType()),
    StructField("p_retailprice", DoubleType()),
    StructField("p_comment", StringType()),
])

PARTSUPP_SCHEMA = StructType([
    StructField("ps_partkey", LongType()),
    StructField("ps_suppkey", LongType()),
    StructField("ps_availqty", LongType()),
    StructField("ps_supplycost", DoubleType()),
    StructField("ps_comment", StringType())
])

REGION_SCHEMA = StructType([
    StructField("r_regionkey", LongType()),   
    StructField("r_name", StringType()),
    StructField("r_comment", StringType()),  
])

SUPPLIER_SCHEMA = StructType([
    StructField("s_suppkey", LongType()),    
    StructField("s_name", StringType()),
    StructField("s_address", StringType()),
    StructField("s_nationkey", LongType()),
    StructField("s_phone", StringType()),
    StructField("s_acctbal", DoubleType()),
    StructField("s_comment", StringType())
])

TABLE_SCHEMA_MAP = {
        "customer": CUSTOMER_SCHEMA,
        "lineitem": LINEITEM_SCHEMA,
        "nation": NATION_SCHEMA,
        "region": REGION_SCHEMA,
        "orders": ORDER_SCHEMA,
        "part": PART_SCHEMA,
        "partsupp": PARTSUPP_SCHEMA,
        "supplier": SUPPLIER_SCHEMA,
}

CURRENT_FILE_PATH = os.path.dirname(os.getcwd())
if "training_data" not in CURRENT_FILE_PATH:
    CURRENT_FILE_PATH += "/training_data"
print(CURRENT_FILE_PATH)
    
def getSystemInfo():
    info={}
    try:
        info['platform']=platform.system()
        info['platform-release']=platform.release()
        info['platform-version']=platform.version()
        info['architecture']=platform.machine()
        info['num_cpus'] = os.cpu_count()
        info['hostname']=socket.gethostname()
        info['ip-address']=socket.gethostbyname(socket.gethostname())
        info['mac-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['processor']=platform.processor()
        info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
        info['total_storage']=str(round(psutil.disk_usage(CURRENT_FILE_PATH).total / (1024.0 **3)))+" GB"
        info['free_storage']=str(round(psutil.disk_usage(CURRENT_FILE_PATH).free / (1024.0 **3)))+" GB"
    except Exception as e:
        logging.exception(e)
    return info

# read in 22 TPCH queries
TPCH_QUERIES = {}
for i in range(1, 23):
    with open(f"{CURRENT_FILE_PATH}/queries/{i}.sql") as f:
        TPCH_QUERIES[i] = f.read() 
    
def get_stats(runtimes):
    print(f'median: {round(np.median(runtimes), 5)}, average: {round(np.average(runtimes), 5)}, std: {round(np.std(runtimes), 5)}, min: {round(min(runtimes), 5)}, max: {round(max(runtimes), 5)}')


/home/hoped/spark-autotuner/training_data


In [2]:
def run_queries_og(parameters, n=10, debug=False, find_median_runtime=True):
    '''
    Run TPC-H queries 10 times and take the median runtime of each query 
    to generate a single training run for a set of parameters.
    
    Input: 
    parameters: list of parameter dictionaries 
    debug: if true will print out params and result time, false suppresses print statements
    
    Returns: 
    training_data dictionary with params and results 
    '''
    result = {'params': [p.copy() for p in parameters], 'runtimes': {'total': []}}
    spark = None
    # add chosen parameter values to spark
    param_name_index = {}
    try:
        conf = SparkConf(loadDefaults=False)
        spark_params = []
        for i, param in enumerate(parameters):
            if param['spark_param']:
                spark_params.append((param['name'], str(param['cur_value'])))
                param_name_index[param['name']] = i

        conf.setAll(spark_params)
        spark  = SparkSession.builder.config(conf=conf).getOrCreate()
                
    except Exception as e:
        if spark:
            spark.stop()
        # this might happen because some parameters are related,
        # and we might have made an impossible parameter assignment
        result = {'params':parameters, 'runtimes': {}, 'msg': str(e)}
        if debug:
            print("error when setting ", parameters, e)
        return result
    
    configurations = spark.sparkContext.getConf().getAll()
    if debug:
        print("Configuration")
    for item in configurations: 
        if debug:
            print(item)
        if param_name_index.get(item[0]) is not None:
            assert item[1] == param[param_name_index.get(item[0])]['cur_value'], f'Spark session param {item} != {param[param_name_index.get(item[0])]}'
    
    # load tables
    if debug:
        print("loading tables")
    tables = {}
    for table_name, table_schema in TABLE_SCHEMA_MAP.items():
        table = spark.read.csv(f"{CURRENT_FILE_PATH}/{SF_STR}/{table_name}.tbl", sep = "|",
                               schema=table_schema)
        table.createOrReplaceTempView(table_name)
        tables[table_name] = table
    
    if debug:
        print("running queries")
    # take median of n runs for each query
    for j in range(n):
        result['runtimes']['total'].append(0)
        for qnum, qtext in TPCH_QUERIES.items(): 
                # Measure execution time of sql query.
            try:
                start_time = time.time()
                results = spark.sql(qtext, **tables)
                end_time = time.time()
                query_time = end_time - start_time
                result['runtimes'].setdefault(qnum, []).append(query_time)
                result['runtimes']['total'][-1] += query_time
            except:
                if debug:
                    print(f"failed while running query {qnum}...  ")
    if debug:
        print("done running queries")
    if find_median_runtime:
        # take median of all runtimes as final output
        for key, times in result['runtimes'].items():
            result['runtimes'][key] = np.median(times)
            if debug:
                print(key, result['runtimes'][key])
    # reset spark so we can load new param config next time
    spark.stop()
    #spark.newSession()#_instantiatedContext attribute of the session to None after calling session.stop().
    
    return result

In [3]:
sf = 1 # GB, default table scale factor
job_name = 'local_run'
SF_STR = f"sf{sf}"

In [18]:
result = run_queries_og([], n=100, find_median_runtime=False)
get_stats(result['runtimes']['total'])
result['runtimes']['total'][0]

median: 0.22342, average: 0.22749, std: 0.03279, min: 0.19532, max: 0.33538


0.22656512260437012

In [19]:
def run_queries(parameters, n=10, find_median_runtime=True):
    '''
    Run TPC-H queries 10 times and take the median runtime of each query 
    to generate a single training run for a set of parameters.
    
    Input: 
    parameters: list of parameter dictionaries 
    n: int number of times to run queries
    find_median_runtime: if True returns runtimes values as a float (median), otherwise as a list of all n runtimes 
    
    Returns: 
    training_data dictionary with params and results 
    '''
    result = {'params': [p.copy() for p in parameters], 'runtimes': {'total': []}}
    spark = None
    # add chosen parameter values to spark
    param_name_index = {}
    try:
        conf = SparkConf(loadDefaults=False)
        spark_params = []
        for i, param in enumerate(parameters):
            if param['spark_param']:
                spark_params.append((param['name'], str(param['cur_value'])))
                param_name_index[param['name']] = i

        conf.setAll(spark_params)
        spark  = SparkSession.builder.config(conf=conf).getOrCreate()
        spark.catalog.clearCache() # clear cache
                
    except Exception as e:
        if spark:
            spark.stop()
        # this might happen because some parameters are related,
        # and we might have made an impossible parameter assignment
        result = {'params':parameters, 'runtimes': {}, 'msg': str(e)}
        return result
    
    configurations = spark.sparkContext.getConf().getAll()
    for item in configurations: 
        if param_name_index.get(item[0]) is not None:
            assert item[1] == param[param_name_index.get(item[0])]['cur_value'], f'Spark session param {item} != {param[param_name_index.get(item[0])]}'
    
    # load tables
    tables = {}
    for table_name, table_schema in TABLE_SCHEMA_MAP.items():
        table = spark.read.csv(f"{CURRENT_FILE_PATH}/{SF_STR}/{table_name}.tbl", sep = "|",
                               schema=table_schema)
        table.createOrReplaceTempView(table_name)
        tables[table_name] = table

    # take median of n runs for each query
    for j in range(n):
        spark.catalog.clearCache() # clear cache before each run
        result['runtimes']['total'].append(0)
        for qnum, qtext in TPCH_QUERIES.items(): 
            # Measure execution time of sql query.
            start_time = time.time()
            results = spark.sql(qtext, **tables)
            end_time = time.time()
            query_time = end_time - start_time
            result['runtimes'].setdefault(qnum, []).append(query_time)
            result['runtimes']['total'][-1] += query_time

    if find_median_runtime:
        # take median of all runtimes as final output
        for key, times in result['runtimes'].items():
            result['runtimes'][key] = np.median(times)

    # reset spark so we can load new param config next time
    spark.catalog.clearCache() # clear cache at the end of each run just in case?
    spark.stop()
    return result

In [17]:
result = run_queries([], n=100, find_median_runtime=False)
get_stats(result['runtimes']['total'])
result['runtimes']['total'][0]

median: 0.22573, average: 0.23069, std: 0.02175, min: 0.1987, max: 0.30497


0.2225637435913086

In [21]:
results_combo = []
for i in range(50):
    result = run_queries_og([], n=50, find_median_runtime=False)
    results_combo.append(result['runtimes']['total'])
first_times = [r[0] for r in results_combo]
all_times = []
for r in results_combo:
    all_times += r

get_stats(first_times)
get_stats(all_times)
print('---')
for r in results_combo:
    get_stats(r)

median: 0.24389, average: 0.24441, std: 0.02324, min: 0.21301, max: 0.30499
median: 0.22808, average: 0.23871, std: 0.02832, min: 0.21113, max: 0.39683
---
median: 0.22931, average: 0.24472, std: 0.03999, min: 0.21149, max: 0.37904
median: 0.21289, average: 0.2282, std: 0.02711, min: 0.21152, max: 0.31709
median: 0.21261, average: 0.22596, std: 0.02791, min: 0.21176, max: 0.31776
median: 0.22027, average: 0.24046, std: 0.03544, min: 0.21211, max: 0.36166
median: 0.23532, average: 0.25024, std: 0.04423, min: 0.21176, max: 0.39683
median: 0.22195, average: 0.24089, std: 0.0348, min: 0.21165, max: 0.33474
median: 0.21418, average: 0.23726, std: 0.03811, min: 0.21195, max: 0.35694
median: 0.22072, average: 0.24038, std: 0.03516, min: 0.2121, max: 0.33191
median: 0.22013, average: 0.24205, std: 0.03588, min: 0.21151, max: 0.35942
median: 0.21381, average: 0.23838, std: 0.03429, min: 0.21165, max: 0.33048
median: 0.23081, average: 0.24055, std: 0.02998, min: 0.2119, max: 0.31352
median: 0.21

In [22]:
results_combo = []
for i in range(50):
    result = run_queries([], n=50, find_median_runtime=False)
    results_combo.append(result['runtimes']['total'])
first_times = [r[0] for r in results_combo]
all_times = []
for r in results_combo:
    all_times += r

get_stats(first_times)
get_stats(all_times)
print('---')
for r in results_combo:
    get_stats(r)

median: 0.2297, average: 0.24443, std: 0.03928, min: 0.21297, max: 0.35476
median: 0.21946, average: 0.24196, std: 0.03845, min: 0.21118, max: 0.74749
---
median: 0.2194, average: 0.23847, std: 0.03307, min: 0.21127, max: 0.3328
median: 0.23846, average: 0.25215, std: 0.0426, min: 0.2116, max: 0.35511
median: 0.22522, average: 0.24008, std: 0.03174, min: 0.2117, max: 0.30977
median: 0.21252, average: 0.22479, std: 0.02312, min: 0.21153, max: 0.29154
median: 0.21275, average: 0.24441, std: 0.08209, min: 0.21151, max: 0.74749
median: 0.21368, average: 0.23668, std: 0.04049, min: 0.21138, max: 0.4195
median: 0.24035, average: 0.24102, std: 0.02753, min: 0.21179, max: 0.30555
median: 0.22158, average: 0.24681, std: 0.04656, min: 0.212, max: 0.4189
median: 0.21265, average: 0.22796, std: 0.02804, min: 0.21147, max: 0.32695
median: 0.24069, average: 0.24752, std: 0.03746, min: 0.21167, max: 0.34528
median: 0.24941, average: 0.25324, std: 0.03403, min: 0.21129, max: 0.33607
median: 0.21306, a

In [27]:
TABLE_FILE_PATH = CURRENT_FILE_PATH + "/../TPC-H V3.0.1/dbgen"
TABLE_FILE_PATH

'/home/hoped/spark-autotuner/training_data/../TPC-H V3.0.1/dbgen'

In [30]:
direc = TABLE_FILE_PATH
files = os.listdir(direc)
files = [f for f in files if os.path.isfile(direc+'/'+f) and '.tbl' in f] #just files
table_sizes = [os.path.getsize(f'{TABLE_FILE_PATH}/{f}') for f in files]
print(round(sum(table_sizes)/ (1024.0 **3)))
num_files = len(files)

files, num_files
    

10


(['supplier.tbl',
  'customer.tbl',
  'orders.tbl',
  'lineitem.tbl',
  'part.tbl',
  'partsupp.tbl',
  'nation.tbl',
  'region.tbl'],
 8)